In [13]:
!pip install nba_api
!pip install keras

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# dependencies

# basic
from datetime import datetime
import json
import pandas as pd
from pandas import ExcelFile
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np
import requests
import seaborn as sns
import time

# nba api
import nba_api
from nba_api.stats.endpoints import BoxScoreAdvancedV2, BoxScoreDefensive, BoxScoreMiscV2, BoxScorePlayerTrackV2, BoxScoreTraditionalV2, BoxScoreUsageV2
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder, leaguegamelog, playergamelog

# sklearn
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, Normalizer, StandardScaler
from sklearn.svm import SVC

# tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

In [2]:
nbateams = teams.get_teams()

In [3]:
df_current_season_games_full = leaguegamelog.LeagueGameLog().get_data_frames()[0]

In [4]:
df_current_season_games_full

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22019,1610612746,LAC,LA Clippers,0021900002,2019-10-22,LAC vs. LAL,W,240,42,...,34,45,24,8,5,14,25,112,10,1
1,22019,1610612747,LAL,Los Angeles Lakers,0021900002,2019-10-22,LAL @ LAC,L,240,37,...,32,41,20,4,7,15,24,102,-10,1
2,22019,1610612761,TOR,Toronto Raptors,0021900001,2019-10-22,TOR vs. NOP,W,265,42,...,41,57,23,7,3,17,24,130,8,1
3,22019,1610612740,NOP,New Orleans Pelicans,0021900001,2019-10-22,NOP @ TOR,L,265,43,...,37,53,30,4,9,19,34,122,-8,1
4,22019,1610612755,PHI,Philadelphia 76ers,0021900008,2019-10-23,PHI vs. BOS,W,240,37,...,50,62,24,6,7,17,34,107,14,1
5,22019,1610612738,BOS,Boston Celtics,0021900008,2019-10-23,BOS @ PHI,L,240,33,...,31,41,18,4,2,11,29,93,-14,1
6,22019,1610612752,NYK,New York Knicks,0021900010,2019-10-23,NYK @ SAS,L,240,44,...,34,39,24,16,1,14,32,111,-9,1
7,22019,1610612759,SAS,San Antonio Spurs,0021900010,2019-10-23,SAS vs. NYK,W,240,42,...,41,51,22,6,5,21,18,120,9,1
8,22019,1610612758,SAC,Sacramento Kings,0021900012,2019-10-23,SAC @ PHX,L,240,34,...,35,47,17,5,5,27,24,95,-29,1
9,22019,1610612756,PHX,Phoenix Suns,0021900012,2019-10-23,PHX vs. SAC,W,240,49,...,35,44,31,13,9,14,25,124,29,1


In [5]:
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result

In [6]:
df_current_season_games = combine_team_games(df_current_season_games_full)

In [7]:
test_array = df_current_season_games['GAME_ID'].unique()[:10]
unique_array = df_current_season_games['GAME_ID'].unique()
test_df = df_current_season_games_full.copy()

In [8]:
df_adv_list = []
df_def_list = []
df_misc_list = []
df_tracking_list = []
df_usage_list = []
df_traditional_list = []

startTime = datetime.now()

game_ok_count = 0
game_no_count = 0

game_problem_list = []

for game in unique_array:
  
    try:

        print(f'Searching for game: {game}...')

        df_advanced = BoxScoreAdvancedV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 1.Advanced df found and saved')
        
        df_defensive = BoxScoreDefensive(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 2.Defensive df found and saved')

        df_misc = BoxScoreMiscV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 3.Misc df found and saved')

        df_tracking = BoxScorePlayerTrackV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 4.Tracking df found and saved')

        df_usage = BoxScoreUsageV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 5.Usage df found and saved')
        
        df_traditional = BoxScoreTraditionalV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 6.Traditional df found and saved')

#         x, = np.where(unique_array == game)
        x, = np.where(test_array == game)
#         x, = np.where(missing_games == game)

        game_ok_count += 1

        print(f'--- {x+1} games searched ---')
        print(f'--- {game_ok_count} games found ---')

        time.sleep(5)

        df_adv_list.append(df_advanced)
        df_def_list.append(df_defensive)
        df_misc_list.append(df_misc)
        df_tracking_list.append(df_tracking)
        df_usage_list.append(df_usage)
        df_traditional_list.append(df_traditional)

        print(f'--- {game_ok_count} games appended ---')

        print(f'--- Time: {datetime.now() - startTime} ---')

    except:
    
    #   problem list
        game_no_count += 1

        game_problem_list.append(game)

        print(f'game {game} encountered timeout')
        print(f'--- {game_no_count} games with problems ---')

    #   sleep
        print(f'--- sleeping for 300 seconds ---')
        time.sleep(300)

    #   try again to search
        try:

            print(f'Searching again for game: {game}...')

            df_advanced = BoxScoreAdvancedV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 1.Advanced df found and saved')
        
            df_defensive = BoxScoreDefensive(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 2.Defensive df found and saved')

            df_misc = BoxScoreMiscV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 3.Misc df found and saved')

            df_tracking = BoxScorePlayerTrackV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 4.Tracking df found and saved')

            df_usage = BoxScoreUsageV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 5.Usage df found and saved')
            
            df_traditional = BoxScoreTraditionalV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 6.Traditional df found and saved')

            x, = np.where(unique_array == game)

            game_ok_count += 1

            print(f'--- {x+1} games searched ---')
            print(f'--- {game_ok_count} games found ---')

            time.sleep(5)

            df_adv_list.append(df_advanced)
            df_def_list.append(df_defensive)
            df_misc_list.append(df_misc)
            df_tracking_list.append(df_tracking)
            df_usage_list.append(df_usage)
            df_traditional_list.append(df_traditional)

            print(f'--- {game_ok_count} games appended ---')

            print(f'--- Time: {datetime.now() - startTime} ---')

        except:

            print(f'game {game} encountered another timeout')
            print(f'--- {game_no_count} games with problems ---')

        #   sleep
            print(f'--- sleeping for 300 seconds ---')
            time.sleep(300)

            continue

    continue

df_adv_all = pd.concat(df_adv_list)
df_def_all = pd.concat(df_def_list)
df_misc_all = pd.concat(df_misc_list)
df_tracking_all = pd.concat(df_tracking_list)
df_usage_all = pd.concat(df_usage_list)
df_traditional_all = pd.concat(df_traditional_list)

# df_def_missing = pd.concat(df_def_list)
# df_misc_missing = pd.concat(df_misc_list)
# df_tracking_missing = pd.concat(df_tracking_list)
# df_usage_missing = pd.concat(df_usage_list)

print(f'--- merged df ---')

Searching for game: 0021900002...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [1] games searched ---
--- 1 games found ---
--- 1 games appended ---
--- Time: 0:00:29.336741 ---
Searching for game: 0021900001...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [2] games searched ---
--- 2 games found ---
--- 2 games appended ---
--- Time: 0:01:00.166488 ---
Searching for game: 0021900008...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [3] games searched ---
--- 3 games found ---
--- 3 games appended ---
--- Time: 0:01:25.708530 ---
Searching for game: 0021900010...
 1.Adv

 6.Traditional df found and saved
--- [] games searched ---
--- 26 games found ---
--- 26 games appended ---
--- Time: 0:12:36.091045 ---
Searching for game: 0021900031...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 27 games found ---
--- 27 games appended ---
--- Time: 0:13:07.532635 ---
Searching for game: 0021900029...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 28 games found ---
--- 28 games appended ---
--- Time: 0:13:39.071465 ---
Searching for game: 0021900035...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
---

 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 52 games found ---
--- 52 games appended ---
--- Time: 0:25:01.267971 ---
Searching for game: 0021900052...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 53 games found ---
--- 53 games appended ---
--- Time: 0:25:30.495497 ---
Searching for game: 0021900053...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 54 games found ---
--- 54 games appended ---
--- Time: 0:25:58.723175 ---
Searching for game: 0021900057...
 1.Advanced df found and saved
 2.Def

--- 77 games appended ---
--- Time: 0:37:09.049198 ---
Searching for game: 0021900079...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 78 games found ---
--- 78 games appended ---
--- Time: 0:37:37.414116 ---
Searching for game: 0021900078...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 79 games found ---
--- 79 games appended ---
--- Time: 0:38:06.910960 ---
Searching for game: 0021900084...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 80 games found ---
--- 80 games appended ---
--- Time: 0

 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 103 games found ---
--- 103 games appended ---
--- Time: 0:49:36.552115 ---
Searching for game: 0021900107...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 104 games found ---
--- 104 games appended ---
--- Time: 0:50:16.015799 ---
Searching for game: 0021900103...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 105 games found ---
--- 105 games appended ---
--- Time: 0:50:47.749151 ---
Searching for game: 0021900104...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved


...

 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 205 games found ---
--- 205 games appended ---
--- Time: 1:39:04.307259 ---
Searching for game: 0021900206...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 206 games found ---
--- 206 games appended ---
--- Time: 1:39:34.213220 ---
Searching for game: 0021900204...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 207 games found ---
--- 207 games appended ---
--- Time: 1:39:59.912594 ---
Searching for game: 0021900208...
 1.Advanced df found and saved


 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 230 games found ---
--- 230 games appended ---
--- Time: 1:51:06.961876 ---
Searching for game: 0021900233...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 231 games found ---
--- 231 games appended ---
--- Time: 1:51:35.020563 ---
Searching for game: 0021900225...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 232 games found ---
--- 232 games appended ---
--- Time: 1:52:02.072014 ---
Searching for game: 0021900226...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.

 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 256 games found ---
--- 256 games appended ---
--- Time: 2:03:27.884927 ---
Searching for game: 0021900253...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 257 games found ---
--- 257 games appended ---
--- Time: 2:03:56.722270 ---
Searching for game: 0021900255...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 258 games found ---
--- 258 games appended ---
--- Time: 2:04:25.494602 ---
Searching for game: 0021900259...
 1.Advanced df found and saved


 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 281 games found ---
--- 281 games appended ---
--- Time: 2:15:12.585944 ---
Searching for game: 0021900283...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 282 games found ---
--- 282 games appended ---
--- Time: 2:15:42.173748 ---
Searching for game: 0021900280...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 283 games found ---
--- 283 games appended ---
--- Time: 2:16:14.532437 ---
Searching for game: 0021900287...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.

 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 307 games found ---
--- 307 games appended ---
--- Time: 2:27:35.612861 ---
Searching for game: 0021900308...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 308 games found ---
--- 308 games appended ---
--- Time: 2:28:04.291737 ---
Searching for game: 0021900307...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [] games searched ---
--- 309 games found ---
--- 309 games appended ---
--- Time: 2:28:28.615021 ---
Searching for game: 0021900315...
 1.Advanced df found and saved


In [10]:
df_adv_all.to_csv('current_advanced.csv')
df_def_all.to_csv('current_defensive.csv')
df_misc_all.to_csv('current_misc.csv')
df_tracking_all.to_csv('current_tracking.csv')
df_usage_all.to_csv('current_usage.csv')
df_traditional_all.to_csv('current_traditional.csv')

In [11]:
stats_df = df_def_all.merge(df_adv_all, 
                            on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [12]:
stats_df2 = stats_df.merge(df_misc_all, on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [13]:
stats_df3 = stats_df2.merge(df_tracking_all, on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [14]:
stats_df4 = stats_df3.merge(df_usage_all, on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [15]:
stats_df5 = stats_df4.merge(df_traditional_all, on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [17]:
stats_df5.sort_index(axis='columns', inplace=True)

In [18]:
stats_df5.to_csv('current_all_boxscores.csv')

In [19]:
list(stats_df5.keys())

['AST_PCT',
 'AST_RATIO',
 'AST_TOV',
 'AST_x',
 'AST_y',
 'BLK',
 'BLKA',
 'BLK_x',
 'BLK_y',
 'CFGA',
 'CFGM',
 'CFG_PCT',
 'COMMENT',
 'DEF_RATING',
 'DFGA',
 'DFGM',
 'DFG_PCT',
 'DIST',
 'DRBC',
 'DREB_PCT',
 'DREB_x',
 'DREB_y',
 'EFG_PCT',
 'E_DEF_RATING',
 'E_NET_RATING',
 'E_OFF_RATING',
 'E_PACE',
 'E_USG_PCT',
 'FG3A',
 'FG3M',
 'FG3_PCT',
 'FGA',
 'FGM',
 'FG_PCT_x',
 'FG_PCT_y',
 'FTA',
 'FTAST',
 'FTM',
 'FT_PCT',
 'GAME_ID',
 'MATCHUP_AST',
 'MATCHUP_FG3A',
 'MATCHUP_FG3M',
 'MATCHUP_FG3_PCT',
 'MATCHUP_FGA',
 'MATCHUP_FGM',
 'MATCHUP_FG_PCT',
 'MATCHUP_MIN',
 'MATCHUP_TOV',
 'MIN',
 'MIN_x',
 'MIN_x',
 'MIN_y',
 'MIN_y',
 'NET_RATING',
 'OFF_RATING',
 'OPP_PTS_2ND_CHANCE',
 'OPP_PTS_FB',
 'OPP_PTS_OFF_TOV',
 'OPP_PTS_PAINT',
 'ORBC',
 'OREB',
 'OREB_PCT',
 'PACE',
 'PACE_PER40',
 'PARTIAL_POSS',
 'PASS',
 'PCT_AST',
 'PCT_BLK',
 'PCT_BLKA',
 'PCT_DREB',
 'PCT_FG3A',
 'PCT_FG3M',
 'PCT_FGA',
 'PCT_FGM',
 'PCT_FTA',
 'PCT_FTM',
 'PCT_OREB',
 'PCT_PF',
 'PCT_PFD',
 'PCT_PT

#### cleanup, add gamescore, etc

In [20]:
df_boxscores = pd.read_csv('current_all_boxscores.csv')

In [21]:
df_boxscores

,AST_PCT,AST_RATIO,AST_TOV,AST,BLK,BLKA,CFGA,CFGM,CFG_PCT,COMMENT,...,TEAM_CITY,TEAM_ID,TEAM_NICKNAME,TM_TOV_PCT,TO,TS_PCT,UFGA,UFGM,UFG_PCT,USG_PCT
0,0.227,14.3,1.67,5,2,1,11,5,0.455,NaN,...,Los Angeles,1610612747,Lakers,8.6,3,0.460,10,3,0.300,0.345
1,0.444,23.5,1.60,8,1,2,8,4,0.500,NaN,...,Los Angeles,1610612747,Lakers,14.7,5,0.434,11,3,0.273,0.302
2,0.000,0.0,0.00,0,2,0,3,2,0.667,NaN,...,Los Angeles,1610612747,Lakers,25.0,1,0.667,0,0,0.000,0.103
3,0.000,0.0,0.00,0,1,0,4,3,0.750,NaN,...,Los Angeles,1610612747,Lakers,0.0,0,0.970,10,7,0.700,0.182
4,0.000,0.0,0.00,0,0,0,3,1,0.333,NaN,...,Los Angeles,1610612747,Lakers,22.2,2,0.571,4,2,0.500,0.170
5,0.125,42.9,3.00,3,0,0,0,0,0.000,NaN,...,Los Angeles,1610612747,Lakers,14.3,1,0.000,3,0,0.000,0.058
6,0.059,25.0,0.00,1,1,2,3,1,0.333,NaN,...,Los Angeles,1610612747,Lakers,0.0,0,0.436,0,0,0.000,0.068
7,0.125,12.5,1.00,1,0,0,1,1,1.000,NaN,...,Los Angeles,1610612747,Lakers,12.5,1,0.466,5,1,0.200,0.206
8,0.167,20.0,2.00,2,0,0,3,1,0.333,NaN,...,Los Angeles,1610612747,Lakers,10.0,1,0.286,-3,-1,0.333,0.182
9,0.000,0.0,0.00,0,0,0,0,0,0.000,NaN,...,Los Angeles,1610612747,Lakers,0.0,0,1.500,2,2,1.000,0.065


In [22]:
list(df_boxscores.keys())

['AST_PCT',
 'AST_RATIO',
 'AST_TOV',
 'AST',
 'BLK',
 'BLKA',
 'CFGA',
 'CFGM',
 'CFG_PCT',
 'COMMENT',
 'DEF_RATING',
 'DFGA',
 'DFGM',
 'DFG_PCT',
 'DIST',
 'DRBC',
 'DREB_PCT',
 'DREB',
 'EFG_PCT',
 'E_DEF_RATING',
 'E_NET_RATING',
 'E_OFF_RATING',
 'E_PACE',
 'E_USG_PCT',
 'FG3A',
 'FG3M',
 'FG3_PCT',
 'FGA',
 'FGM',
 'FG_PCT',
 'FTA',
 'FTAST',
 'FTM',
 'FT_PCT',
 'GAME_ID',
 'MATCHUP_AST',
 'MATCHUP_FG3A',
 'MATCHUP_FG3M',
 'MATCHUP_FG3_PCT',
 'MATCHUP_FGA',
 'MATCHUP_FGM',
 'MATCHUP_FG_PCT',
 'MATCHUP_MIN',
 'MATCHUP_TOV',
 'MIN',
 'NET_RATING',
 'OFF_RATING',
 'OPP_PTS_2ND_CHANCE',
 'OPP_PTS_FB',
 'OPP_PTS_OFF_TOV',
 'OPP_PTS_PAINT',
 'ORBC',
 'OREB',
 'OREB_PCT',
 'PACE',
 'PACE_PER40',
 'PARTIAL_POSS',
 'PASS',
 'PCT_AST',
 'PCT_BLK',
 'PCT_BLKA',
 'PCT_DREB',
 'PCT_FG3A',
 'PCT_FG3M',
 'PCT_FGA',
 'PCT_FGM',
 'PCT_FTA',
 'PCT_FTM',
 'PCT_OREB',
 'PCT_PF',
 'PCT_PFD',
 'PCT_PTS',
 'PCT_REB',
 'PCT_STL',
 'PCT_TOV',
 'PFD',
 'PF',
 'PIE',
 'PLAYER_ID',
 'PLAYER_NAME',
 'PLAYE

In [23]:
df_boxscores['GAMESCORE'] = 0
for index,row in df_boxscores.iterrows():
    
    pts = df_boxscores.loc[index, 'PTS']
    fgm = df_boxscores.loc[index, 'FGM']
    oreb = df_boxscores.loc[index, 'OREB']
    dreb = df_boxscores.loc[index, 'DREB']
    stl = df_boxscores.loc[index, 'STL']
    ast = df_boxscores.loc[index, 'AST']
    blk = df_boxscores.loc[index, 'BLK']
    fga = df_boxscores.loc[index, 'FGA']
    fta = df_boxscores.loc[index, 'FTA']
    ftm = df_boxscores.loc[index, 'FTM']
    ftmiss = fta - ftm
    pf = df_boxscores.loc[index, 'PF']
    tov = df_boxscores.loc[index, 'TO']
    
    gamescore = round((
            (pts) +
            (0.4*fgm) +
            (0.7*oreb) +
            (0.3*dreb) +
            (stl) +
            (0.7*ast) +
            (0.7*blk) -
            (0.7*fga) -
            (0.4*ftmiss) -
            (0.4*pf) -
            (tov)
    ), 3)
    
    print(gamescore)
    
    df_boxscores.loc[index, 'GAMESCORE'] = gamescore
    

17.1
11.6
4.1
26.2
2.0
-2.1
4.1
1.9
0.6
4.6
20.7
2.4
6.7
5.5
2.6
13.3
20.2
8.0
12.1
5.9
15.7
4.6
9.2
7.1
13.7
6.6
3.6
-0.4
4.7
12.4
4.5
6.7
25.4
0.8
14.8
32.2
6.7
2.3
6.5
17.2
10.0
9.6
4.3
-1.2
4.5
2.9
-0.4
0.6
-1.2
-0.7
4.7
13.8
9.6
13.3
13.0
19.9
-2.3
1.3
2.8
4.0
4.7
0.7
0.0
23.9
18.9
8.4
-3.1
15.4
-0.9
-1.9
7.2
-0.1
15.5
-2.1
5.1
16.4
7.5
15.8
19.0
5.5
10.9
7.0
1.6
3.1
3.8
9.3
1.7
16.6
0.3
-4.4
-0.4
8.0
3.8
5.6
2.3
0.9
0.3
16.5
5.4
19.3
17.3
13.3
9.1
1.4
6.0
7.8
8.3
1.2
-0.7
2.7
7.1
9.6
31.4
7.5
10.5
2.8
3.5
1.3
11.9
8.3
9.8
15.7
8.9
10.2
40.5
0.4
0.7
0.8
3.0
-1.9
10.6
12.0
15.2
5.4
6.1
4.0
3.8
1.4
-2.3
3.9
6.2
18.1
10.5
8.7
8.8
3.2
8.5
4.3
1.2
9.4
8.5
0.6
3.2
5.3
8.9
11.8
3.6
0.1
4.8
0.6
1.6
20.8
11.3
-1.1
9.9
17.1
14.9
0.4
6.9
-0.9
8.2
-0.7
0.0
1.9
17.4
-0.9
13.2
17.6
-0.0
3.0
-2.3
5.7
3.4
9.7
12.1
3.3
27.9
-5.7
2.7
10.7
2.0
1.3
0.3
18.8
10.7
14.3
-2.6
10.1
8.7
1.8
7.3
10.1
4.0
7.0
3.4
17.0
0.0
24.9
-2.1
4.7
7.3
0.3
0.7
6.1
2.0
36.7
3.8
6.1
23.4
15.5
1.7
2.9
-1.0
-0.7
2.0
24.2
20.

22.0
7.8
19.5
20.7
-0.0
13.7
13.2
12.9
7.6
4.0
-1.5
4.5
17.0
10.4
11.9
8.4
8.3
3.0
-3.0
3.8
0.3
21.6
-0.1
8.4
13.4
16.1
12.8
-0.0
5.7
7.2
2.3
0.3
2.6
19.9
8.9
30.3
8.9
1.5
3.2
10.7
0.6
6.2
29.1
10.6
9.3
15.4
9.7
5.2
16.5
7.3
13.8
15.2
27.4
-0.1
8.7
-1.7
5.3
-3.1
4.6
0.7
7.9
0.5
15.6
31.7
20.1
-0.8
6.1
1.6
0.3
2.8
10.3
31.0
3.1
12.3
11.1
8.5
-1.2
4.9
5.0
-1.4
5.2
1.9
13.9
11.5
16.1
15.1
34.6
15.5
2.2
5.6
-1.5
22.0
30.0
10.7
6.6
18.7
3.1
-0.3
1.2
13.8
4.6
-0.9
0.6
0.3
3.9
11.1
11.4
13.4
6.7
8.5
-0.1
3.0
0.2
1.6
11.7
3.1
-0.6
14.0
16.1
13.9
13.9
4.4
7.4
-1.5
-1.7
0.0
4.3
11.6
9.3
14.6
8.7
6.1
17.2
2.6
12.8
9.2
-0.1
-0.7
11.0
-1.0
10.9
4.9
30.9
2.1
6.4
0.2
5.7
-0.8
3.5
5.3
7.9
11.4
18.3
11.6
-1.3
9.3
-1.5
13.4
30.5
10.8
14.5
8.2
4.5
4.7
16.8
3.4
2.6
5.3
0.0
11.5
2.4
7.1
21.4
9.7
13.2
0.5
9.5
6.5
2.7
0.0
1.0
11.2
37.1
4.8
7.7
17.0
-1.5
0.3
8.3
3.1
11.7
-0.5
8.3
15.6
19.3
13.7
-2.7
8.2
14.4
3.0
1.1
4.5
19.4
11.7
11.4
8.7
-1.8
3.9
4.8
16.0
-1.7
1.1
-2.7
12.4
14.6
1.5
9.2
24.4
3.4
2.3
0.6
4.1


35.3
8.7
6.5
14.1
8.8
11.5
12.0
6.9
-1.3
3.5
1.0
0.3
28.3
15.7
8.8
1.5
13.1
11.6
9.7
4.2
5.5
3.8
0.6
0.7
-1.7
1.4
8.2
11.6
9.8
4.1
0.6
4.3
-3.6
14.3
5.7
0.2
4.7
1.4
4.5
13.9
24.4
10.6
10.0
13.0
15.3
5.3
2.4
8.3
33.2
23.7
12.9
12.0
9.7
-1.5
0.1
4.3
10.0
10.2
13.3
2.6
14.2
-1.1
0.5
9.4
0.5
11.4
8.5
10.6
14.3
6.4
28.7
0.3
12.1
2.7
0.5
-2.9
3.6
5.4
23.9
9.6
4.2
37.9
9.4
11.2
7.3
13.4
-2.7
1.0
-1.1
0.3
7.6
14.7
11.8
0.8
7.9
16.2
4.4
6.3
0.5
5.2
2.9
11.6
24.2
4.3
11.5
10.1
2.2
-4.1
14.6
9.2
1.6
4.3
5.1
11.0
19.6
17.1
11.9
13.0
-0.7
-1.7
-0.4
18.9
21.0
17.3
6.7
11.3
5.2
12.3
1.3
3.4
2.7
3.1
0.8
5.0
3.6
0.4
7.7
13.3
20.2
9.6
7.7
1.5
9.7
3.8
3.3
5.7
11.9
8.9
3.6
5.0
10.3
-0.9
20.2
27.9
7.5
10.8
5.2
6.7
4.9
6.3
2.3
6.1
2.9
9.2
19.2
1.6
15.4
9.6
1.8
11.0
3.3
1.4
2.2
0.7
7.5
5.8
9.2
10.2
4.6
1.8
3.5
4.7
6.0
-1.1
4.9
10.6
14.9
6.0
16.3
23.2
4.9
0.5
9.6
5.8
3.4
0.0
2.1
0.7
11.3
8.7
21.5
1.6
-0.3
6.3
1.0
-1.2
0.0
1.2
-0.8
5.4
5.6
11.0
-0.1
22.4
1.8
14.2
7.0
-2.2
-0.4
7.3
2.2
10.9
7.5
-0.8
0.2
9.4
-2.

0.7
4.4
-0.4
1.0
2.1
7.6
1.8
21.2
11.3
3.0
52.4
12.5
6.5
12.0
4.6
9.0
10.0
9.0
2.0
-1.5
19.1
15.1
8.3
-2.5
7.0
0.4
0.1
4.8
10.6
21.8
13.8
8.8
9.8
11.6
11.7
9.9
8.5
11.9
4.2
8.5
2.2
9.6
1.8
4.0
22.2
10.8
7.6
2.2
-2.5
6.5
9.7
21.5
7.1
12.6
14.1
5.5
3.1
-0.3
7.1
2.9
4.9
8.5
11.6
3.1
21.0
22.4
-1.2
6.9
-0.1
10.7
2.3
20.5
21.1
7.2
3.5
4.4
-2.4
8.0
4.9
1.9
4.2
2.4
1.3
21.1
25.0
2.2
-5.7
10.9
11.9
1.4
3.0
11.4
0.9
4.9
-2.3
7.4
15.3
5.9
9.9
-0.4
2.9
11.0
13.9
6.7
0.4
13.7
13.2
9.2
8.8
4.5
17.6
5.0
5.0
0.5
-0.7
0.0
17.9
5.3
6.5
3.7
18.7
1.4
15.2
-0.1
6.8
2.2
5.5
24.1
5.2
17.5
11.1
18.6
0.8
0.7
8.6
2.7
28.4
5.8
-1.4
21.7
7.1
18.5
20.6
4.5
8.1
5.2
3.9
2.0
-2.1
11.8
12.9
12.4
12.3
10.8
6.9
4.3
-4.3
4.8
12.3
1.8
11.5
21.9
15.7
2.3
-1.6
-0.4
14.1
0.0
16.8
6.1
11.3
6.3
2.3
8.3
-0.1
-0.4
8.6
5.2
10.8
1.7
2.1
5.6
11.5
17.1
12.2
7.3
12.3
7.8
9.8
27.5
1.7
1.7
3.0
9.6
-1.9
4.6
17.9
12.3
9.6
8.4
12.2
10.6
3.9
3.0
8.0
15.3
9.2
14.6
12.6
4.7
2.8
7.1
3.5
5.1
7.4
9.6
10.9
15.3
6.2
-4.3
3.0
15.2
5.0
-0.9
3.2
1.

In [24]:
df_boxscores.to_csv('current_all_boxscores.csv')

## player selection